In [27]:
import requests
import numpy as np
from utils import *

In [28]:
topics_list = []
number_of_pages = 100
URLs = [
    "https://community.jenkins.io/c/using-jenkins/support/8.json"
]

for page in URLs:
    
    for page_num in range(number_of_pages):   
             
        # URL to make the GET request to
        url = f"{page}?page={page_num}"

        # Make the GET request
        response = requests.get(url)

        if response.status_code == 200:
            # Parse the JSON response into a Python dictionary
            json_response = response.json()
            
            
            # Extract posts from the JSON object
            topics = json_response["topic_list"]["topics"]
            # print(page_num, len(topics))
            
            # Find the post with accepted_answer = true
            for topic in topics:
                if topic.get("has_accepted_answer"):
                    topics_list.append(topic)

        else:
            # Print an error message if the request was not successful
            print("Error:", response.status_code)


In [29]:
print(len(topics_list))

344


In [30]:
# Get the topics IDs
topics_ids = [topic["id"] for topic in topics_list]

In [31]:
accepted_answers = []
original_questions = []

for topic_id in topics_ids:
    
    # URL to make the GET request to
    url = f"https://community.jenkins.io/t/{topic_id}.json?track_visit=true&forceLoad=true"

    # Make the GET request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary
        json_response = response.json()
        
        
        # Extract posts from the JSON object
        posts = json_response["post_stream"]["posts"]
        # print(posts)
        
        # Find the post with accepted_answer = true
        for post in posts:
            if post.get("accepted_answer"):
                original_questions.append(posts[0]['cooked'])
                accepted_answers.append(post['cooked'])
                break
        else:
            print(f"No accepted answer found for topic {topic_id}")

    else:
        # Print an error message if the request was not successful
        print("Error:", response.status_code)


No accepted answer found for topic 11272


In [32]:
print(len(original_questions), len(accepted_answers))

343 343


In [33]:
combined_arrays = np.column_stack((original_questions, accepted_answers))

df = pd.DataFrame(combined_arrays, columns=['questions', 'answers'])

df['questions'] = df['questions'].apply(remove_html_tags)
df['answers'] = df['answers'].apply(remove_html_tags)

In [34]:
df.to_csv('../datasets/Community Questions Refined.csv', index=False, encoding='utf-8', date_format=str)

In [18]:
# import csv

# rows = list(zip(original_questions, accepted_answers))

# with open('Community Questions.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['Question', 'Answer'])  # Write header
    
#     for row in rows:
#         writer.writerow(row)

# print("CSV file with arrays saved as 'special_characters.csv'")


CSV file with arrays saved as 'special_characters.csv'


In [53]:
# import json
# # Write posts data to a JSON file
# with open("show.json", "w") as json_file:
#     json.dump(posts, json_file, indent=4)

In [54]:
# accepted_answers[0]